<h1>Capstone Project <h1>
<h2>[Sistem Klasifikasi Harga Smartphone Berdasarkan Spesifikasi dan Ekspansinya]<h2>
<h4>Kelas : M2<h4>
<h4>Tean Id : CSD-186<h4>

## Business Understanding

Semakin banyaknya perusahaan-perusahaan pembuat produk handphone atau smartphone, membuat produsen serta konsumen harus dapat menyesuaikan produk terhadap finansial. Apa yang menjadi tolak ukur utama dalam pendistribusian produk? Siapa yang terlibat dalam penyesuaian finansial produk? Mengapa finansial menjadi tolak ukur utama dalam produksi? Kapan produk harus diberi kesesuaian finansial? Di mana produk harus diberi kesesuaian finansial? Bagaimana permasalahan tersebut dapat terselesaikan? Proyek ini bertujuan untuk membuat sebuah web aplikasi dengan fitur sistem klasifikasi harga berdasarkan spesifikasi dan ekspansi berupa sistem rekomendasi berdasarkan harga. Kami memilih proyek ini dikarenakan permasalahan yang digunakan cukup umum dan memandang fungsionalitas proyek yang efektif.

### Problem Statement
Berapa Harga Smartphone yang cocok dengan spesifikasi yang diinginkan pengguna?

### Goals
Menentukan Harga Smartphone yang cocok dengan spesifikasi yang diinginkan pengguna?

### Solution statements

Solusi yang diajukan terdiri dari 3 jenis model, antara lain KNN, Random Forest, dan AdaBoost, yang kemudian dipilih dari antara ketiga model tersebut, yang memiliki tingkat akurasi paling tinggi.

- **K-Nearest Neighbours**. Merupakan algoritma machine learning untuk proses klasifikasi. Algoritma ini merupakan *supervised learning* dan mengelompokan suatu label dengan cara mencari kesesuaian dengan tetangga terdekat.

- **AdaBoost**. atau adaptive boosting merupakan salah satu teknik ensamble dengan menggunakan loss function fungsi exponential untuk memperbaiki tingkat akurasi dari prediksi yang dibuat.

- **Random Forest**. Random forest merupakan kombinasi dari masing–masing pohon dari model Decision Tree yang baik, dan kemudian dikombinasikan ke dalam satu model. Penentuan klasifikasi dengan random forest diambil berdasarkan hasil voting dari tree yang terbentuk.

#### Import Seluruh Library Dependencies

In [1]:
import zipfile
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn import metrics
from sklearn.svm import SVC

# Import library Model
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import AdaBoostClassifier

## Data Loading

Install library kaggle dan membuat directory baru bernama kaggle

In [2]:
! pip install kaggle
! cp kaggle.json ~/kaggle/

'cp' is not recognized as an internal or external command,
operable program or batch file.


Copy "kaggle.json" ke dalam direktori yang telah dibuat, kemudian Mengalokasikan izin yang diperlukan untuk file tersebut.

In [3]:
! chmod 600 ~/kaggle/kaggle.json

# Mendownload dan menyiapkan dataset 
! kaggle datasets download iabhishekofficial/mobile-price-classification

chmod: cannot access '/root/.kaggle/kaggle.json': No such file or directory
Traceback (most recent call last):
  File "/usr/local/bin/kaggle", line 5, in <module>
    from kaggle.cli import main
  File "/usr/local/lib/python2.7/dist-packages/kaggle/__init__.py", line 23, in <module>
    api.authenticate()
  File "/usr/local/lib/python2.7/dist-packages/kaggle/api/kaggle_api_extended.py", line 146, in authenticate
    self.config_file, self.config_dir))
IOError: Could not find kaggle.json. Make sure it's located in /root/.kaggle. Or use the environment method.


Mendownload dataset mobile price classification.

In [4]:
# Mengekstrak zip file

local_zip = '/content/mobile-price-classification.zip'
zip_ref = zipfile.ZipFile(local_zip, 'r')
zip_ref.extractall('/content')
zip_ref.close()

FileNotFoundError: ignored

Dataset yang digunakan berupa informasi mengenai estimasi kelompok harga dari beberapa smartphone atau mobile devices berdasarkan spesifikasi yang dimiliki perangkat tersebut, seperti RAM, prosesor, dan lainnya, Dataset:
[Mobile Price Classification](https://www.kaggle.com/iabhishekofficial/mobile-price-classification)

Melakukan ekstraksi pada file dengan ekstensi zip agar dataset yang diperlukan dapat digunakan.

In [ ]:
# Mengubah dataset menjadi dataframe

df = pd.read_csv('/content/train.csv')
df.head()

## Data Understanding

Data yang digunakan dalam proyek ini adalah dataset [*Mobile Price Classification*](https://www.kaggle.com/iabhishekofficial/mobile-price-classification) yang didapatkan melalui situs kaggle. Dataset ini terdiri dari beberapa variabel pendukung seperti kapasitas baterai, RAM, prosesor dan masih banyak lagi. Dataset ini terdiri dari 2000 sampel data dan 21 kolom dengan 1 kolom target prediksi yaitu price range atau kelompok harga dari smartphone. Kolom pertama hingga kolom ke-dua puluh satu merupakan data numerik.

Deskripsi variabel:
1. battery_power = energi total yang dapat disimpan baterai dalam satu waktu diukur dalam mAh
2. blue = ada tidaknya bluetooth
3. clock_speed = kecepatan di mana mikroprosesor mengeksekusi instruksi
4. dual_sim = mendukung dual sim atau tidak
5. fc = front camera dalam mega pixels
6. four_g = mendukung 4G atau tidak
7. int_memory = kapasitas internal memori dalam Gigabytes
8. m_dep = kedalaman hp dalam cm
9. mobile_wt = berat hp 
10. n_cores = jumlah cores dari processor
11. pc = kamera utama dalam mega pixels
12. px_height = tinggi resolusi pixel
13. px_width = lebar resolusi pixel
14. ram = jumlah ram dalam Mega Bytes
15. sc_h = tinggi layar ponsel dalam cm
16. sc_w = lebar layar ponsel dalam cm
17. talk_time = waktu terlama satu kali pengisian baterai akan bertahan saat terakhir kali Anda berada
18. three_g = mendukung 3G atau tidak
19. touch_screen = mendukung layar sentuh atau tidak
20. wifi = mendukung wifi atau tidak
21. price_range = variabel target dengan nilai 0 (biaya rendah), 1 (biaya sedang), 2 (biaya tinggi) dan 3 (biaya sangat tinggi)

In [ ]:
# Melihat informasi mengenai dataset
df.info()

Melihat informasi statistik yang ada pada dataset. Terlihat bahwasanya dataset memiliki 2000 sampel.

In [ ]:
df.describe()

Melihat deskripsi statistik pada dataset. Dapat menjadi catatan, bahwasanya tidak akan rasional apabila nilai pada px_height, px_width, sc_h, sc_w sama dengan 0. Karena nilai-nilai yang ada pada variabel tersebut merepresentasikan dimensi layar hp.

In [ ]:
# Melihat apakah ada missing value pada dataset
df.isnull().sum()

Melihat apakah ada atau tidaknya missing value pada dataset. Terlihat bahwasanya tidak ada missing value pada dataset.

In [ ]:
# Memeriksa rasionalitas beberapa variabel
px_h = (df.px_height==0).sum()
px_w = (df.px_width==0).sum()
sc_h = (df.sc_h==0).sum()
sc_w = (df.sc_w==0).sum()

Memeriksa rasionalitas beberapa variabel, hal ini diperlukan untuk melihat ada atau tidaknya nilai pada variabel yang tidak rasional.

In [ ]:
print(px_h)
print(px_w)
print(sc_h)
print(sc_w)

In [ ]:
# Menghapus nilai yang tidak rasional pada variabel
df = df.loc[(df[['px_height','sc_w']]!=0).all(axis=1)]
df.shape

Maka tersisa 1819 sampel setelah menghilangkan sampel yang mengandung variabel dengan nilai yang tidak rasional.

### Menangani Outliers

Setelah itu menangani Outliers dengan IQR Method dan menggunakan boxplot untuk visualisasi pada tiap variabel

In [ ]:
sns.boxplot(x=df['battery_power'])

Melihat ada tidaknya outliers dengan IQR Method dan menggunakan boxplot untuk visualisasi pada variabel battery_power.

In [ ]:
sns.boxplot(x=df['blue'])

Melihat ada tidaknya outliers dengan IQR Method dan menggunakan boxplot untuk visualisasi pada variabel blue.

In [ ]:
sns.boxplot(x=df['clock_speed'])

Melihat ada tidaknya outliers dengan IQR Method dan menggunakan boxplot untuk visualisasi pada variabel clock_speed.

In [ ]:
sns.boxplot(x=df['dual_sim'])

Melihat ada tidaknya outliers dengan IQR Method dan menggunakan boxplot untuk visualisasi pada variabel dual_sim.

In [ ]:
sns.boxplot(x=df['fc'])

Melihat ada tidaknya outliers dengan IQR Method dan menggunakan boxplot untuk visualisasi pada variabel fc.

In [ ]:
sns.boxplot(x=df['four_g'])

Melihat ada tidaknya outliers dengan IQR Method dan menggunakan boxplot untuk visualisasi pada variabel four_g.

In [ ]:
sns.boxplot(x=df['int_memory'])

Melihat ada tidaknya outliers dengan IQR Method dan menggunakan boxplot untuk visualisasi pada variabel int_memory.

In [ ]:
sns.boxplot(x=df['m_dep'])

Melihat ada tidaknya outliers dengan IQR Method dan menggunakan boxplot untuk visualisasi pada variabel m_dep.

In [ ]:
sns.boxplot(x=df['mobile_wt'])

Melihat ada tidaknya outliers dengan IQR Method dan menggunakan boxplot untuk visualisasi pada variabel mobile_wt.

In [ ]:
sns.boxplot(x=df['n_cores'])

Melihat ada tidaknya outliers dengan IQR Method dan menggunakan boxplot untuk visualisasi pada variabel n_cores.

In [ ]:
sns.boxplot(x=df['pc'])

Melihat ada tidaknya outliers dengan IQR Method dan menggunakan boxplot untuk visualisasi pada variabel pc.

In [ ]:
sns.boxplot(x=df['px_height'])

Melihat ada tidaknya outliers dengan IQR Method dan menggunakan boxplot untuk visualisasi pada variabel px_height.

In [ ]:
sns.boxplot(x=df['px_width'])

Melihat ada tidaknya outliers dengan IQR Method dan menggunakan boxplot untuk visualisasi pada variabel px_width.

In [ ]:
sns.boxplot(x=df['ram'])

Melihat ada tidaknya outliers dengan IQR Method dan menggunakan boxplot untuk visualisasi pada variabel ram.

In [ ]:
sns.boxplot(x=df['sc_h'])

Melihat ada tidaknya outliers dengan IQR Method dan menggunakan boxplot untuk visualisasi pada variabel sc_h.

In [ ]:
sns.boxplot(x=df['sc_w'])

Melihat ada tidaknya outliers dengan IQR Method dan menggunakan boxplot untuk visualisasi pada variabel sc_w.

In [ ]:
sns.boxplot(x=df['talk_time'])

Melihat ada tidaknya outliers dengan IQR Method dan menggunakan boxplot untuk visualisasi pada variabel talk_time.

In [ ]:
sns.boxplot(x=df['three_g'])

Melihat ada tidaknya outliers dengan IQR Method dan menggunakan boxplot untuk visualisasi pada variabel three_g.

In [ ]:
sns.boxplot(x=df['touch_screen'])

Melihat ada tidaknya outliers dengan IQR Method dan menggunakan boxplot untuk visualisasi pada variabel touch_screen.

In [ ]:
sns.boxplot(x=df['wifi'])

Melihat ada tidaknya outliers dengan IQR Method dan menggunakan boxplot untuk visualisasi pada variabel wifi.

In [ ]:
# Proses drop outliers
Q1 = df.quantile(0.25)
Q3 = df.quantile(0.75)
IQR=Q3-Q1
df=df[~((df<(Q1-1.5*IQR))|(df>(Q3+1.5*IQR))).any(axis=1)]
 
# Cek ukuran dataset setelah outliers didrop
df.shape

Melakukan drop outliers. Terdapat 1368 sampel setelah melakukan drop pada sampel yang memiliki outliers.

### Visualisasi Data
Visualisasi yang digunakan adalah berupa histogram pada masing-masing Kolom pada dataset

In [ ]:
# Melakukan proses analisis data dengan teknik Univariate EDA
import matplotlib.pyplot as plt
%matplotlib inline

df.hist(bins=50, figsize=(20,15))
plt.show()

In [ ]:
l_three = (df.price_range==3).sum()
l_two = (df.price_range==2).sum()

print(l_three)
print(l_two)

Dari histogram di atas, terdapat informasi bahwasanya pada tiap label/target price_range jumlahnya tidak berbeda secara signifikan, dengan kata lain, hal tersebut bisa dikatakan proporsional. Jumlah tertinggi ada pada target 3 (biaya sangat tinggi).